In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

print("ready!")

ready!

In [2]:
!wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz -P /tmp
dbutils.fs.mv("file:/tmp/kddcup.data_10_percent.gz", "dbfs:/kdd/kddcup_data.gz")

display(dbutils.fs.ls("dbfs:/kdd"))

In [3]:
data_file = "dbfs:/kdd/kddcup_data.gz"
raw_rdd = sc.textFile(data_file).cache()

raw_rdd.take(5)

Out[10]: ['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In [4]:
raw_rdd.count()

Out[11]: 494021

In [5]:
csv_rdd = raw_rdd.map(lambda row: row.split(","))
len(csv_rdd.take(1)[0])

Out[20]: 42

In [6]:
from pyspark.sql import Row

parsed_rdd = csv_rdd.map(lambda r: Row(
    duration=int(r[0]),
    protocol_type=r[1],
    service=r[2],
    flag=r[3],
    src_bytes=int(r[4]),
    dst_bytes=int(r[5]),
    wrong_fragment=int(r[7]),
    urgent=int(r[8]),
    hot=int(r[9]),
    num_failed_logins=int(r[10]),
    num_compromised=int(r[12]),
    su_attempted=r[14],
    num_root=int(r[15]),
    num_file_creations=int(r[16]),
    label=r[-1]
    )
)
parsed_rdd.take(5)

Out[21]: [Row(dst_bytes=5450, duration=0, flag='SF', hot=0, label='normal.', num_compromised=0, num_failed_logins=0, num_file_creations=0, num_root=0, protocol_type='tcp', service='http', src_bytes=181, su_attempted='0', urgent=0, wrong_fragment=0),
 Row(dst_bytes=486, duration=0, flag='SF', hot=0, label='normal.', num_compromised=0, num_failed_logins=0, num_file_creations=0, num_root=0, protocol_type='tcp', service='http', src_bytes=239, su_attempted='0', urgent=0, wrong_fragment=0),
 Row(dst_bytes=1337, duration=0, flag='SF', hot=0, label='normal.', num_compromised=0, num_failed_logins=0, num_file_creations=0, num_root=0, protocol_type='tcp', service='http', src_bytes=235, su_attempted='0', urgent=0, wrong_fragment=0),
 Row(dst_bytes=1337, duration=0, flag='SF', hot=0, label='normal.', num_compromised=0, num_failed_logins=0, num_file_creations=0, num_root=0, protocol_type='tcp', service='http', src_bytes=219, su_attempted='0', urgent=0, wrong_fragment=0),
 Row(dst_bytes=2032, duration=0, flag='SF', hot=0, label='normal.', num_compromised=0, num_failed_logins=0, num_file_creations=0, num_root=0, protocol_type='tcp', service='http', src_bytes=217, su_attempted='0', urgent=0, wrong_fragment=0)]

In [7]:
spark = sc.getOrCreate()
type(spark)

Out[26]: __main__.RemoteContext

In [8]:
df = parsed_rdd.toDF()
display(df.head(10))

dst_bytes,duration,flag,hot,label,num_compromised,num_failed_logins,num_file_creations,num_root,protocol_type,service,src_bytes,su_attempted,urgent,wrong_fragment
5450,0,SF,0,normal.,0,0,0,0,tcp,http,181,0,0,0
486,0,SF,0,normal.,0,0,0,0,tcp,http,239,0,0,0
1337,0,SF,0,normal.,0,0,0,0,tcp,http,235,0,0,0
1337,0,SF,0,normal.,0,0,0,0,tcp,http,219,0,0,0
2032,0,SF,0,normal.,0,0,0,0,tcp,http,217,0,0,0
2032,0,SF,0,normal.,0,0,0,0,tcp,http,217,0,0,0
1940,0,SF,0,normal.,0,0,0,0,tcp,http,212,0,0,0
4087,0,SF,0,normal.,0,0,0,0,tcp,http,159,0,0,0
151,0,SF,0,normal.,0,0,0,0,tcp,http,210,0,0,0
786,0,SF,1,normal.,0,0,0,0,tcp,http,212,0,0,0


In [9]:
df.printSchema()

root
-- dst_bytes: long (nullable = true)
-- duration: long (nullable = true)
-- flag: string (nullable = true)
-- hot: long (nullable = true)
-- label: string (nullable = true)
-- num_compromised: long (nullable = true)
-- num_failed_logins: long (nullable = true)
-- num_file_creations: long (nullable = true)
-- num_root: long (nullable = true)
-- protocol_type: string (nullable = true)
-- service: string (nullable = true)
-- src_bytes: long (nullable = true)
-- su_attempted: string (nullable = true)
-- urgent: long (nullable = true)
-- wrong_fragment: long (nullable = true)

In [11]:
df.registerTempTable("connections")

In [12]:
x = df.groupBy("protocol_type") \
      .count() \
      .orderBy("count", ascending=True)
display(x)

protocol_type,count
udp,20354
tcp,190065
icmp,283602


In [13]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

protocols = sqlContext.sql("""
                           SELECT protocol_type, count(*) as freq
                           FROM connections
                           GROUP BY protocol_type
                           ORDER BY 2 DESC
                           """)
display(protocols)

protocol_type,freq
icmp,283602
tcp,190065
udp,20354


In [14]:
labels = sqlContext.sql("""
                           SELECT label, count(*) as freq
                           FROM connections
                           GROUP BY label
                           ORDER BY 2 DESC
                           """)
display(labels)

label,freq
smurf.,280790
neptune.,107201
normal.,97278
back.,2203
satan.,1589
ipsweep.,1247
portsweep.,1040
warezclient.,1020
teardrop.,979
pod.,264


In [15]:
type(labels)

Out[38]: pyspark.sql.dataframe.DataFrame

In [16]:
labels_df = pd.DataFrame(labels.toPandas())
labels_df.set_index("label", drop=True,inplace=True)
labels_fig = labels_df.plot(kind='barh')

plt.rcParams["figure.figsize"] = (7, 5)
plt.rcParams.update({'font.size': 10})
plt.tight_layout()
display(labels_fig.figure)